In [5]:
import numpy as np
import pandas as pd

# Get the data

In [6]:
col_names= ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]

data=pd.read_csv("C:\\Users\\Utkarsh\Downloads\data\iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


In [7]:
# There are three types of irsir flowers.

data["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

# Object oriented programming will be used here.

# Node class

In [63]:
class Node():
    def __init__(self, feature_index=None, Threshold=None, left=None, right=None, info_gain=None, value=None):
        '''Constructor'''
                
        #for decision node
        self.feature_index= feature_index
        self.Threshold= Threshold
        self.left= left
        self.right= right
        self.info_gain= info_gain        
                
        # for leaf node
        self.value= value
            

# Tree class

In [74]:
class DecisionTreeClassifier():
    
    def __init__(self, min_samples_split=2, max_depth=2):
        '''Constructor'''
        
        #initialize the root of the tree
        self.root= None
        
        #stopping conditions
        
        #minimum required items in every node after a split
        self.min_samples_split = min_samples_split
        
        #max depth of the tree
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        '''recursive fucntion to build the tree'''
        
        X, Y= dataset[:,:-1], dataset[:,-1]
        num_samples, num_features= np.shape(X)
        
        if num_samples>=self.min_samples_split and curr_depth<= self.max_depth:
            
            #find the best split
            best_split= self.get_best_split(dataset, num_samples, num_features)
            
            #Checking if the info gain is +ve
            if(best_split["info_gain"]>0):
                
                #left tree
                left_subtree= self.build_tree(self, best_split["dataset_left"], curr_depth+1)
                
                #right tree
                right_subtree= self.build_tree(self, best_split["dataset_right"], curr_depth+1)
                
                #return decision node
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree,
                            right_subtree, best_split["info_gain"])
            
            #calculate leaf node
            leaf_value= self.calculate_leaf_value(Y)
            
            #return leaf node
            return(Node(value=leaf_value))
                    
    def get_best_split(self, dataset, num_samples, num_features):        
        '''Function to find the best split'''
        
        #dictionary to store the best split
        best_split = {}
        
        '''-ve infinity is initialised. This is because we want to maximise the information gain and 
        to do that we need a number less than any other number.'''
        max_info_gain = -float("inf")
        
        
        '''Features are real numbers and they are infinite between any two numbers. Sp to find the threshold we 
        can't iterate infinite times. Instead we ietrate over tha values of features we have in our dataset'''
        for feature_index in range(num_features):
            
            feature_values= dataset[:, feature_index]
            possible_thresholds= np.unique(feature_values)
            
            for threshold in possible_thresholds:
                
                #get current split 
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                #Check if the child are null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:,-1], dataset[:,-1], dataset[:,-1]
                    
                    #compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"]=feature_index
                        best_split["threshold"]=threshold
                        best_split["dataset_left"]=dataset_left
                        best_split["dataset_right"]=dataset_right
                        best_split["info_gain"]= curr_info_gain
                        max_info_gain = curr_info_gain
                        
            #return best split
            return best_split
    
    def split(self, dataset, feature_index, threshold):
        '''function to split the data'''
        
        dataset_left= np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right= np.array([row for row in dataset if row[feature_index]>threshold])
        
        return dataset_left, dataset_right
                        
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        '''function to compute information gain'''
        
        weight_l= len(l_child)/len(parent)
        weight_r= len(r_child)/len(parent)
        
        if mode=="gini":
            gain= self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))            
        return gain
            
    
    def entropy(self, y):
        '''function to compute entropy'''
        
        class_labels= np.unique(y)
        entropy =0
        for cls in class_labels:
            p_cls = len(y[y==cls])/ len(y)
            entropy += -p_cls * np.log2(p_cls)
                        
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    
    def calculate_leaf_value(self, Y):
        '''function to cmppute leaf node'''
        
        Y= list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        '''function to print the tree'''
        
        if not tree:
            tree= self.root
            
        if tree.value is not None:
            print(tree.value)
            
        else:
        
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            
    def fit(self, X, Y):
        
        dataset= np.concatenate((X,Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def predict(self, X):
        '''function to predict new dataset'''
        
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        
        if tree.value!= None:
            return tree.value
        
        feature_val= x[tree.feature_index]
        
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
                

# Train test split

In [75]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

# Fit the model

In [76]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()

Iris-setosa


# Test the model

In [78]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.3